## Outils pour la manipulation d'images et librairies.


In [20]:
import PIL
from PIL import Image
import numpy as np
import scipy as sp
import os
from math import log10, sqrt

def load(filename):
    toLoad= Image.open(filename)
    return np.asarray(toLoad)


def psnr(original, compressed):
    mse = np.mean((original - compressed) ** 2)
    max_pixel = 255.0
    psnr = 20 * log10(max_pixel / sqrt(mse))
    return psnr

def dct2(a):
    return sp.fft.dct( sp.fft.dct( a, axis=0, norm='ortho' ), axis=1, norm='ortho' )

def idct2(a):
    return sp.fft.idct( sp.fft.idct( a, axis=0 , norm='ortho'), axis=1 , norm='ortho')



## Normalisation de l'image (YCbCr et padding)

Question 1 : Donner le code qui transforme une image RGB en une image YCbCr. Vous pourrez produire
une matrice pour chaque composante, pour pouvoir plus facilement les manipuler indépendamment. Vous
pouvez stocker les données YCbCr comme des entiers ou des flottants, mais vous expliquerez votre choix.

In [21]:
def RGB_en_YCbCR(image):
    image_finale = np.array(image, dtype='float64')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            R = image[i, j, 0]
            G = image[i, j, 1]
            B = image[i, j, 2]
            image_finale[i, j, 0] = 0.299 * R + 0.587 * G + 0.114 * B
            image_finale[i, j, 1] = -0.1687 * R - 0.3313 * G + 0.5 * B + 128
            image_finale[i, j, 2] = 0.5 * R - 0.4187 * G - 0.0813 * B + 128
    return image_finale


Question 2 : Donner le code qui transforme une image YCbCr en une image RGB. Attention, les valeurs des
canaux RGB doivent être un entier dans [0, 255] qui pourra être codé sur un octet. Appliquer successivement
la transformation RGB vers YCbCr puis YCbCr vers RGB et vérifier que vous obtenez l’image de départ.
Vous consulterez la documentation des fonctions de numpy clip, uint8 et mask qui pourraient vous être
utiles.

In [22]:
def YCbCR_en_RGB(image):
    image_finale = np.array(image, dtype='uint8')
    for i in range(image.shape[0]):
        for j in range(image.shape[1]):
            Y = image[i, j, 0]
            Cb = image[i, j, 1]
            Cr = image[i, j, 2]
            image_finale[i, j, 0] = Y + 1.402 * (Cr - 128)
            image_finale[i, j, 1] = Y - 0.34414 * (Cb - 128) - 0.71414 * (Cr - 128)
            image_finale[i, j, 2] = Y + 1.772 * (Cb - 128)
    return np.uint8(np.clip(image_finale, 0, 255))


Question 3 : Donner la fonction qui réalise ce padding ainsi que celle qui l’élimine et vérifier que
l’application de ces deux transformations laissent l’image inchangée.

In [23]:

def padding(image):
    padded_image = np.array(image, dtype='uint8')
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]
    canaux = padded_image.shape[2]
    if ligne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne%8, colonne, canaux), dtype=np.uint8)), axis=0)
    if colonne % 8 != 0:
        padded_image = np.concatenate((padded_image, np.zeros((ligne, colonne%8, canaux), dtype=np.uint8)), axis=1)
    return padded_image

In [24]:
def unpadding(padded_image, image_origine):
    ligne = padded_image.shape[0]
    colonne = padded_image.shape[1]

    ligne_origine = image_origine.shape[0]
    colonne_origine = image_origine.shape[1]

    # Calculer le nombre de colonnes à enlever
    nb_col_enlever = colonne - colonne_origine
    while nb_col_enlever > 0:
        padded_image = padded_image[:, :-1]
        nb_col_enlever -= 1

    # Calculer le nombre de lignes à enlever
    nb_ligne_enlever = ligne- ligne_origine
    while nb_ligne_enlever > 0:
        padded_image = padded_image[:-1, :]
        nb_ligne_enlever -= 1

    return padded_image

Question 4 : Implémenter la fonction qui sous-échantillonne une matrice et renvoie une matrice deux fois
plus petite.

In [25]:
def sous_echantillonnage(matrice):
    matrice2 = matrice.copy()
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1]):
            matrice[i, j, 1] = 0
            matrice[i, j, 2] = 0
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1], 2):
            new_valueCb = (matrice2[i][j][1] + matrice2[i][j+1][1])//2
            new_valueCr = (matrice2[i][j][2] + matrice2[i][j+1][2])//2
            matrice[i, j//2, 1] = new_valueCb
            matrice[i, j//2, 2] = new_valueCr
    return matrice

Question 5 : Implémenter la fonction qui multiplie par deux la deuxième dimension d’une matrice. Tester à la suite le sous-échantillonnage et cette fonction, vous devez retrouver une image presque identique à celle de départ.

In [26]:

def mult_mat(matrice):
    matrice2 = matrice.copy()
    for i in range(matrice.shape[0]):
        for j in range(0, matrice.shape[1]//2):
            matrice[i][j*2][1] = matrice2[i][j][1]
            matrice[i][j*2][2] = matrice2[i][j][2]
            matrice[i][j*2+1][1] = matrice2[i][j][1]
            matrice[i][j*2+1][2] = matrice2[i][j][2]
    return matrice

## Découpage en blocs et compression

Question 6 : Soit une matrice dont les deux dimensions sont divisibles par 8. Donner une fonction qui découpe cette matrice en blocs 8 × 8 et les stocke dans une liste. L’ordre des blocs correspond à l’ordre de lecture d’une image.

In [29]:
def decoupe_bloc(matrice):
    lignes = matrice.shape[0]
    colonnes = matrice.shape[1]
    blocs = []
    
    # Parcours des indices de début de chaque bloc
    for i in range(0, lignes, 8):
        for j in range(0, colonnes, 8):
            # Création d'un nouveau bloc vide
            bloc = []
            
            # Parcours des lignes du bloc
            for k in range(i, i + 8):
                # Extraction des éléments de la ligne correspondante
                ligne = matrice[k][j:j+8]
                # Ajout de la ligne au bloc
                bloc.append(ligne)
            
            # Ajout du bloc à la liste
            blocs.append(bloc)
    
    return blocs

[[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16]
 [ 17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32]
 [ 33  34  35  36  37  38  39  40  41  42  43  44  45  46  47  48]
 [ 49  50  51  52  53  54  55  56  57  58  59  60  61  62  63  64]
 [ 65  66  67  68  69  70  71  72  73  74  75  76  77  78  79  80]
 [ 81  82  83  84  85  86  87  88  89  90  91  92  93  94  95  96]
 [ 97  98  99 100 101 102 103 104 105 106 107 108 109 110 111 112]
 [113 114 115 116 117 118 119 120 121 122 123 124 125 126 127 128]]
[[array([1, 2, 3, 4, 5, 6, 7, 8]), array([17, 18, 19, 20, 21, 22, 23, 24]), array([33, 34, 35, 36, 37, 38, 39, 40]), array([49, 50, 51, 52, 53, 54, 55, 56]), array([65, 66, 67, 68, 69, 70, 71, 72]), array([81, 82, 83, 84, 85, 86, 87, 88]), array([ 97,  98,  99, 100, 101, 102, 103, 104]), array([113, 114, 115, 116, 117, 118, 119, 120])], [array([ 9, 10, 11, 12, 13, 14, 15, 16]), array([25, 26, 27, 28, 29, 30, 31, 32]), array([41, 42, 43, 44, 45, 46, 47, 48]), 

Question 7 : Donner une fonction qui applique la transformée à chaque bloc d’une liste

In [ ]:
def applique_trans(liste_bloc):
    for i in range(len(liste_bloc)):
        liste_bloc[i] = dct2(liste_bloc[i])
    return liste_bloc

Question 8 : Implémentez le filtrage des coefficients des blocs selon un seuil donné en argument.

## Écriture dans un fichier

In [1]:
#ecrire dans un fichier

def ecrire_fichier(liste_bloc, fichier):
    with open(fichier, 'w') as f:
        for bloc in liste_bloc:
            for ligne in bloc:
                for element in ligne:
                    f.write(str(element) + ' ')
                f.write('\n')
            f.write('\n')

#lire dans un fichier
def lire_fichier(fichier):
    with open(fichier, 'r') as f:
        lignes = f.readlines()
        blocs = []
        bloc = []
        for ligne in lignes:
            if ligne == '\n':
                blocs.append(bloc)
                bloc = []
            else:
                ligne = ligne.split(' ')
                ligne = [int(element) for element in ligne if element != '']
                bloc.append(ligne)
    return blocs

## Décompression

## Tests 

In [28]:
test = load("test.png")
testYCbCr = YCbCR_en_RGB(RGB_en_YCbCR(test))
#Image.fromarray(test,'RGB').show()
#Image.fromarray(testYCbCr,'RGB').show()

padded_image = padding(testYCbCr)

#Image.fromarray(padded_image,'RGB').show()
#Image.fromarray(unpadding(padded_image, test),'RGB').show()

Image.fromarray(YCbCR_en_RGB(mult_mat(sous_echantillonnage(RGB_en_YCbCR(test)))),'RGB').show()
print(psnr(test, YCbCR_en_RGB(mult_mat(sous_echantillonnage(RGB_en_YCbCR(test))))))

43.60365608263673
